<a id="top"></a>
#### QCT. Practices with Adiabatic Quantum Computing

# Quadratic Knapsack Problems

#### Lecturers:
- Rafael Martín-Cuevas Redondo, rafamartinc@pm.me

***

In this notebook, we will explore how to implement a sample Quadratic Knapsack Problem, and find its solution using the solvers provided by D-Wave Systems.

### Table of Contents

1. [Shaping the QUBO Model](#qubo_model)
2. [Finding the solution with D-Wave's solvers](#solution_dwave)

***

Please run the following block to import the libraries that will be required during the execution of this notebook. If your system lacks any of the libraries mentioned, use ```pip install -r requirements.txt``` in a command line, within this notebook's folder, to ensure that all required libraries are installed.

In [1]:
from pyqubo import Array, Placeholder, Constraint

import neal

[Back to the top](#top)

***
<a id="qubo_model"></a>

## 1. Shaping the QUBO Model

We will start by modeling the Hamiltonian described during the lecture. During this notebook, we will use the library PyQUBO to define our QUBO model.

In [2]:
num_variables = 4

max_weight = 6

profits = [
    [3, 1, 0, -3],
    [0, 7, 0,  4],
    [0, 0, 2,  0],
    [0, 0, 0, 10]
]

weights = [2, 1, 3, 4]

x = Array.create('x', shape=num_variables, vartype='BINARY')

In [3]:
# First part of the Hamiltonian.
H_linear_profit = 0

for i in range(num_variables):
    H_linear_profit += Constraint(
        profits[i][i] * x[i], label='profit({})'.format(i)
    )

In [4]:
# Second part of the Hamiltonian.
H_quadratic_profit = 0

for i in range(num_variables):
    for j in range(i + 1, num_variables):
        H_quadratic_profit += Constraint(
            profits[i][j] * x[i] * x[j], label='profit({}, {})'.format(i, j)
        )

In [5]:
# Third part of the Hamiltonian.
H_linear_weight = 0

for i in range(num_variables):
    H_linear_weight += Constraint(weights[i] * x[i], label='weight({})'.format(i))

In [6]:
# Define full Hamiltonian.
alpha = Placeholder('alpha')
H = - H_linear_profit - H_quadratic_profit + alpha * (max_weight - H_linear_weight)**2

print(H)

((Placeholder('alpha') * ((6.000000 + (-1.000000 * (Constraint((4.000000 * Binary('x[3]')), 'weight(3)') + Constraint((3.000000 * Binary('x[2]')), 'weight(2)') + Constraint((1.000000 * Binary('x[1]')), 'weight(1)') + 0.000000 + Constraint((2.000000 * Binary('x[0]')), 'weight(0)')))) * (6.000000 + (-1.000000 * (Constraint((4.000000 * Binary('x[3]')), 'weight(3)') + Constraint((3.000000 * Binary('x[2]')), 'weight(2)') + Constraint((1.000000 * Binary('x[1]')), 'weight(1)') + 0.000000 + Constraint((2.000000 * Binary('x[0]')), 'weight(0)')))))) + (-1.000000 * (Constraint((10.000000 * Binary('x[3]')), 'profit(3)') + Constraint((2.000000 * Binary('x[2]')), 'profit(2)') + Constraint((7.000000 * Binary('x[1]')), 'profit(1)') + 0.000000 + Constraint((3.000000 * Binary('x[0]')), 'profit(0)'))) + (-1.000000 * (Constraint(((0.000000 * Binary('x[2]')) * Binary('x[3]')), 'profit(2, 3)') + Constraint(((4.000000 * Binary('x[1]')) * Binary('x[3]')), 'profit(1, 3)') + Constraint(((0.000000 * Binary('x[1]

In [7]:
# Compile model
model = H.compile()

# Create QUBO with alpha = 100.0
feed_dict = {'alpha': 100.0}
qubo, offset = model.to_qubo(feed_dict=feed_dict)

print(qubo)

{('x[2]', 'x[0]'): 1200.0, ('x[3]', 'x[0]'): 1603.0, ('x[1]', 'x[1]'): -1107.0, ('x[3]', 'x[1]'): 796.0, ('x[3]', 'x[3]'): -3210.0, ('x[1]', 'x[0]'): 399.0, ('x[2]', 'x[1]'): 600.0, ('x[3]', 'x[2]'): 2400.0, ('x[0]', 'x[0]'): -2003.0, ('x[2]', 'x[2]'): -2702.0}


We have now expressed our QUBO model as a data structure that can be recognized by our solvers.

[Back to the top](#top)

***
<a id="solution_dwave"></a>

## 2. Finding the solution with D-Wave's solvers

Finally, we can use the solvers provided by D-Wave to find the solution to this problem, as follows.

In [8]:
sampler = neal.SimulatedAnnealingSampler()
num_reads = 5000

response = sampler.sample_qubo(qubo, num_reads=num_reads)
response = response.aggregate()

for values, _, num_occurrences in response.data():
    variables = [key for key in values if values[key] != 0]
    print('Solution {}: {}/{} occurrences'.format(
        variables, num_occurrences, num_reads
    ))

Solution ['x[0]', 'x[1]', 'x[2]']: 1898/5000 occurrences
Solution ['x[0]', 'x[3]']: 2235/5000 occurrences
Solution ['x[0]', 'x[1]', 'x[3]']: 217/5000 occurrences
Solution ['x[1]', 'x[3]']: 197/5000 occurrences
Solution ['x[2]', 'x[3]']: 353/5000 occurrences
Solution ['x[0]', 'x[2]']: 100/5000 occurrences


In [9]:
# Get best solution.

best_solution = None
best_solution_occurrences = -1

for values, _, num_occurrences in response.data():
    if num_occurrences > best_solution_occurrences:
        best_solution = values
        best_solution_occurrences = num_occurrences

best_solution_variables = [
    key for key in best_solution
    if best_solution[key] != 0 and key[:2] == 'x['
]

print('Best solution ({:0.1f}%): {}'.format(
    100 * best_solution_occurrences / num_reads,
    best_solution_variables
))

Best solution (44.7%): ['x[0]', 'x[3]']


In [10]:
# Check weight constraint.

best_solution_weights = [
    weights[int(key[2:-1])] for key in best_solution_variables
]
total_weight = sum(best_solution_weights)

print('\nTotal investment: {} <= {} - {}'.format(
    total_weight,
    max_weight,
    'OK!' if total_weight <= max_weight else 'ERROR'
))
for key in best_solution_variables:
    asset_id = int(key[2:-1])
    print('   {}: {}'.format(key, weights[asset_id]))

print('\n   Cash: {}'.format(max_weight - total_weight))


Total investment: 6 <= 6 - OK!
   x[0]: 2
   x[3]: 4

   Cash: 0


[Back to the top](#top)